In [2]:
import pandas as pd
import json
import sqlite3
import os

In [10]:
OUTPUT_PATH = "extracted_chats.json"

In [ ]:
conn = sqlite3.connect("/opt/openwebui/data/webui.db")
cur = conn.cursor()
emails = pd.read_csv("data/test_emails.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'test_emais.csv'

In [12]:
'''
OUTPUT_JSON_SHAPE = {
    "user_id" : "",
    "name" : "" ,
    "email": "" ,
    "role" : "" ,
    "chats" : [ {
        "title" : "" ,
        "message_pairs" : [
            {
                "timestamp": "...",
                "question" : "...",
                "answer" : "..." 
            } ,  
            ... 
            {

            }
        ]
    ]
}
'''


'\nOUTPUT_JSON_SHAPE = {\n    "user_id" : "",\n    "name" : "" ,\n    "email": "" ,\n    "role" : "" ,\n    "chats" : [ {\n        "title" : "" ,\n        "message_pairs" : [\n            {\n                "timestamp": "...",\n                "question" : "...",\n                "answer" : "..." \n            } ,  \n            ... \n            {\n\n            }\n        ]\n    ]\n}\n'

In [5]:
conn.close()

In [ ]:
all_users = []

for email in emails['email']:
    # Fetch user
    cur.execute("SELECT id, name, role FROM user WHERE email = ?", (email,))
    val = cur.fetchone()
    if not val:
        print(f"No user found for {email}")
        exit()

    user_id, name, role = val

    json_structure = {
        "user_id": user_id,
        "name": name,
        "email": email,
        "role": role,
        "chats": []
    }

    # Fetch all chats for this user
    cur.execute("SELECT chat FROM chat WHERE user_id = ?", (user_id,))
    chat_rows = cur.fetchall()
    for idx, row in enumerate(chat_rows):
        chat_file = row[0]
        chat_json = json.loads(chat_file)

        chat_entry = { 
            "title": chat_json.get('title', 'Unknown'),
            "message_pairs": []
        }

        messages = chat_json.get("messages", [])
        for j in range(0, len(messages), 2):
            ts = messages[j].get('timestamp', 0)
            q = messages[j].get("content", "")
            a = messages[j+1].get("content", "") if j+1 < len(messages) else None
            chat_entry["message_pairs"].append({"timestamp": ts, "question": q, "answer": a})

        json_structure["chats"].append(chat_entry)

    all_users.append(json_structure)

# add to json file
with open(OUTPUT_PATH, "w") as f:
    json.dump(all_users, f, indent=4)


print(json.dumps(json_structure, indent=2))

{
  "user_id": "8f214f5d-caf4-44de-bf33-52752b6da6a6",
  "name": "Lihang Liu",
  "email": "lliu432@wisc.edu",
  "role": "admin",
  "chats": [
    {
      "title": "\u5bc6\u5c14\u6c83\u57fa\u5bb6\u5ead\u4ecb\u7ecd \ud83d\udc4b",
      "message_pairs": [
        {
          "timestamp": 1762124006,
          "question": "\u4f60\u597d",
          "answer": "\u4f60\u597d\uff0c\u6211\u662fTony Beckham\uff0c\u4f4f\u5728\u5bc6\u5c14\u6c83\u57fa\u3002\u548c\u6211\u592a\u592aAntonella\u4e00\u8d77\uff0c\u6709\u4e24\u4e2a\u5b69\u5b50Alana\u548cBradley\uff0c\u8fd8\u6709\u4e00\u4e2a\u5c0f\u5b59\u5b50Max\u3002\u6709\u4ec0\u4e48\u6211\u53ef\u4ee5\u5e2e\u5fd9\u56de\u7b54\u7684\u5417\uff1f"
        }
      ]
    },
    {
      "title": "\ud83d\udc4b Meet Tony Beckham",
      "message_pairs": [
        {
          "timestamp": 1762125677,
          "question": "What is your name!!! say it out loud!!!",
          "answer": "I'm Tony Beckham."
        },
        {
          "timestamp": 1762125715,
      